In [ ]:
import os
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm
from scipy.stats import spearmanr
from statsmodels.stats.multitest import multipletests
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer

In [ ]:

# data_folder = '/home/jaume/Desktop/Data/New_ACDC/MIDS/mixed/derivatives/'
derivatives_path = "/media/jaume/DATA/Data/Urblauna_SFTP/UKB_Cardiac_BIDS/derivatives"
experiment_name = 'GraphClassification'

# Get the list of subjects
biomarkers_filename = os.path.join(derivatives_path, 'biomarkers.csv')
df_biomarkers = pd.read_csv(biomarkers_filename, index_col=0)

metadata_filename = os.path.join(derivatives_path, 'metadata_participants_numeric.csv')
df_numeric_metadata = pd.read_csv(metadata_filename, index_col=0)

merged_filename = os.path.join(derivatives_path, 'metadata_participants_biomarkers.csv')
df_merged = pd.read_csv(merged_filename, index_col=0)

In [ ]:
# ==== UKB
derivatives_folder = "/media/jaume/DATA/Data/Urblauna_SFTP/UKB_Cardiac_BIDS/derivatives"
data_path = os.path.join(derivatives_folder, 'GraphClassification')
all_edges = 'Edges-True_Norm-ZNorm_Global-True_All-True_Sim-False_BP-False'
aha_edges = 'Edges-True_Norm-ZNorm_Global-True_All-False_Sim-False_BP-False'
# study_name = 'Multiplex_HPT_ACDC_ADAM_FINAL_MAE'

# =========================== ALL ===========================
study_name = 'Multiplex_HPT_UKB_DIMENSIONS_NEW_LOSS_ALL'
all_data_folder = os.path.join(data_path, all_edges, study_name)  # save_folder

# =========================== AHA ===========================
study_name = 'Multiplex_HPT_UKB_DIMENSIONS_NEW_LOSS'
aha_data_folder = os.path.join(data_path, aha_edges, study_name)  # save_folder

save_folder = os.path.join(all_data_folder, 'latent_analysis') 
latent_filename = os.path.join(all_data_folder, 'latent_data.csv')
save_folder = os.path.join(aha_data_folder, 'latent_analysis') 
latent_filename = os.path.join(aha_data_folder, 'latent_data.csv')
os.makedirs(save_folder, exist_ok=True)



In [ ]:
df_latent = pd.read_csv(latent_filename)
df_latent['Subject'] = df_biomarkers['Subject'].copy()
drop_columns=['Region', 'Cycle', 'dt', 'ed_cycle_time', 'es_cycle_time', 'ed_frame_idx', 'es_frame_idx', 'Height']

In [ ]:
df_merged.drop(columns=['Group'], inplace=True)
drop_columns_biomarkers = ['LV_SV', 'LV_Myo_EF', 'LV_Myo_SV', 'RV_SV', 'RV_Myo_SV', 'RV_Myo_EF', 'RV_Myo_SVI', 'LV_Myo_SVI']
df_merged[['RVMI']] = df_merged[['RV_Myo_SVI']] * 1.05
df_merged[['LVMI']] = df_merged[['LV_Myo_SVI']] * 1.05

In [ ]:
# Prepare data for the regression analysis
y_df = df_merged.copy()
y_df.drop(columns=drop_columns, inplace=True)  # Remove non-feature columns
y_df.drop(columns=drop_columns_biomarkers, inplace=True)
y_df.set_index('Subject', inplace=True)

missing_counts = y_df.isna().sum()
print(list(missing_counts[missing_counts > 0].sort_values().index))

y_df = y_df.loc[:, y_df.isnull().mean() < 0.3]
# imputer = KNNImputer(n_neighbors=5)
# y_df_imputed = pd.DataFrame(imputer.fit_transform(y_df), columns=y_df.columns, index=y_df.index)
imputer = IterativeImputer(random_state=42)
y_df_imputed = pd.DataFrame(imputer.fit_transform(y_df), columns=y_df.columns, index=y_df.index)
feature_names = y_df.columns.tolist()
y_df = y_df_imputed.copy()

In [ ]:
# Prepare data for the regression analysis
X_df = df_latent.copy()
X_df.drop(columns=['Sample', 'labels'], inplace=True)  # Remove non-feature columns
X_df.set_index('Subject', inplace=True)

In [ ]:
# Step 1: PCA on X_df === Too much otherwise
X_df_clean = X_df.dropna(axis=1, thresh=int(0.9 * len(X_df)))  # optional: remove columns with too many NaNs
X_df_filled = X_df_clean.fillna(X_df_clean.mean())  # mean impute if necessary

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_df_filled)

pca = PCA()
X_pca_full = pca.fit_transform(X_scaled)

# Step 2: Choose number of components explaining target variance (e.g., 95%)
explained_var_ratio = np.cumsum(pca.explained_variance_ratio_)
n_components = np.argmax(explained_var_ratio >= 0.95) + 1

print(f"Selected {n_components} components to explain 95% of the variance.")

X_pca = pd.DataFrame(X_pca_full[:, :n_components],
                     index=X_df.index,
                     columns=[f"PC{i+1}" for i in range(n_components)])

# Step 3: Join with y_df
df_joined = X_pca.join(y_df, how='inner')

In [ ]:

# Step 4: Compute Spearman correlations: each PC vs each Y variable
results = []
for pc in X_pca.columns:
    for yvar in y_df.columns:
        if pc not in df_joined.columns or yvar not in df_joined.columns:
            continue
        subset = df_joined[[pc, yvar]].dropna()
        if len(subset) < 10:
            continue
        x = subset[pc]
        y = subset[yvar]
        if x.nunique() < 2 or y.nunique() < 2:
            continue
        r, p = spearmanr(x, y)
        results.append({'PC': pc, 'Y_var': yvar, 'Correlation': r, 'p_value': p})

assoc_df = pd.DataFrame(results)

# Step 4: FDR correction (Benjamini-Hochberg)
pvals = assoc_df['p_value'].values
rejected, pvals_corrected, _, _ = multipletests(pvals, method='fdr_bh', alpha=0.05)
assoc_df['p_value_corrected'] = pvals_corrected
assoc_df['Significant'] = rejected
assoc_df['Significant_NC'] = pvals < 0.05

In [ ]:
assoc_df.sort_values(by='p_value', ascending=True)

In [ ]:
# Params
n_jobs = 3
n_permutations = 200
alphas=[0.01, 0.05, 0.1, 0.5, 1, 5, 10]

def permutation_test(target):
    y = y_df[target].values.squeeze()
    r2_permute = []
    X_used = X_pca.copy()

    # Rigid    
    search = GridSearchCV(make_pipeline(StandardScaler(), Ridge()), 
                          param_grid={'ridge__alpha': alphas}, scoring='r2', cv=5, n_jobs=3)
    search.fit(X_used, y)
    final_params = {key.split('__')[1]: value for key, value in search.best_params_.items()}

    # Fit model
    model = make_pipeline(StandardScaler(), Ridge(**final_params))
    model.fit(X_used, y)
    y_pred = model.predict(X_used)
    r2_train = r2_score(y, y_pred)

    # Permutation loop
    for _ in range(n_permutations):
        y_perm = np.random.permutation(y.squeeze())
        model_perm = make_pipeline(StandardScaler(), Ridge(**final_params))
        model_perm.fit(X_used, y_perm)
        r2_permute.append(r2_score(y_perm, model_perm.predict(X_used)))

    # Compute p-value
    p_value = (1 + np.sum(np.array(r2_permute) >= r2_train)) / (1 + len(r2_permute))


    return {
        'target': target,
        'r2_train': r2_train,
        'r2_perm_mean': np.mean(r2_permute),
        'r2_perm_std': np.std(r2_permute),
        'r2_permute_all': r2_permute,  # full list
        'p_value': p_value,
    }

# Run in parallel
results = Parallel(n_jobs=n_jobs)(
    delayed(permutation_test)(target) for target in tqdm(y_df.columns, desc="Permutation test")
)

# Summary table
summary_df = pd.DataFrame({
    res['target']: {
        'R2_train': res['r2_train'],
        'R2_perm_mean': res['r2_perm_mean'],
        'R2_perm_std': res['r2_perm_std'],
        'p_value': res['p_value']
    }
    for res in results
}).T

# Full permutation R2s (for plotting)
r2_perm_full_df = pd.DataFrame({
    res['target']: pd.Series(res['r2_permute_all'])  # pad shorter series with NaN if needed
    for res in results
})

# Save
summary_df.to_csv(f"{save_folder}/rf_permutation_summary_pca.csv")
r2_perm_full_df.to_csv(f"{save_folder}/rf_permutation_r2_distributions_pca.csv")

In [ ]:
from statsmodels.stats.multitest import multipletests

# Apply multiple testing correction (FDR)
summary_df["fdr_corrected_p"] = multipletests(summary_df["p_value"], method="fdr_bh")[1]
summary_df["sig_marker"] = summary_df["p_value"].apply(lambda p: "*" if p < 0.1 else "")
summary_df.loc[summary_df["fdr_corrected_p"] < 0.1, "sig_marker"] = "**"
summary_df

# Compute values for volcano plot
summary_df["r2_diff"] = summary_df["R2_train"] - summary_df["R2_perm_mean"]
summary_df["neg_log_p"] = -np.log10(summary_df["p_value"])

# Plot
plt.figure(figsize=(10, 6))
colors = summary_df["p_value"] < 0.1
plt.scatter(summary_df["r2_diff"], summary_df["neg_log_p"], c=colors.map({True: "red", False: "gray"}))
plt.axhline(-np.log10(0.1), linestyle="--", color="black", linewidth=1)
plt.xlabel("ΔR² (Train - Permuted Mean)")
plt.ylabel("-log₁₀(p-value)")
plt.title("Volcano Plot: Predictive Gain vs. Statistical Significance")

# Annotate top features (optional)
top = summary_df[summary_df["p_value"] < 0.1].sort_values("neg_log_p", ascending=False).head(5)
for idx, row in top.iterrows():
    plt.text(row["r2_diff"], row["neg_log_p"] + 0.1, idx, fontsize=8, ha='center')

plt.grid(True)
plt.tight_layout()
plt.show()

summary_df.sort_values(by='p_value', ascending=True)


In [ ]:
assoc_df.to_csv(os.path.join(save_folder, 'association_ukb_pc.csv'))
assoc_df.sort_values(by='p_value', ascending=True)
assoc_df